In [3]:
from MOF_build import buildMIL53tetra

In [17]:
x_num,y_num,z_num,x_scalar,y_scalar,z_scalar =10,5,5,3.6,16,16
outnode, outlinker, outcut, outall = ( "M2node", "M2linker", "M2cut", "M2all")  # name of output file
O1_index, O2_index, O3_index, O4_index = 49,56,59,51
linker_pdb = '../linkers4demo/r_M2.pdb'

In [18]:
x_num,y_num,z_num,x_scalar,y_scalar,z_scalar =10,5,5,3.6,16,16
outnode, outlinker, outcut, outall = ( "M4node", "M4linker", "M4cut", "M4all")  # name of output file
O1_index, O2_index, O3_index, O4_index = 49,56,59,51#49,56,59,51
linker_pdb = '../linkers4demo/r_M4.pdb'

In [19]:
mof = buildMIL53tetra.MIL53tetra(
    linker_pdb, O1_index, O2_index, O3_index,O4_index,
    x_num, y_num, z_num,
    x_scalar, y_scalar, z_scalar, 
)
mof.buildall(outnode, outcut, outlinker, outall)
mof.view(outall)


pointsA number:  198
pointsB number:  198
allpoints number:  396
['AL6', 'MOH', 'CUT', 'TCC'] [7, 2, 7, 88]
total atoms: 16608
AL6   396
MOH   396
CUT   292
TCC   125

Time cost (s):   0:06:49.365300


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [13]:
from MDprepare import move2C
import py3Dmol as p3d
move2C.atoms2c('centerTCC',1.5,2,2,7,12,12) 

In [18]:
from MDprepare import move2C
import py3Dmol as p3d
move2C.atoms2c('all',1.5,2,2,7,12,12) 

#!gmx solvate -cp newall.gro -o sol_all.gro #99171 water


def viewgro(groname):
    viewer = p3d.view(width=600, height=600)
    viewer.addModelsAsFrames(open(groname+'.gro', "r").read(), "gro", {"keepH": True})
    viewer.setStyle({"stick": {}, "sphere": {"scale": 0.25}})
    viewer.zoomTo()
    viewer.show()

#viewgro('sol_all')
''' 
insert 1000 water molecules in the box before MD running
'''
a = 'MDdemo/mdfiles'
mdp = 'MDdemo/mdp'
itp = 'MDdemo/MILitp'
!gmx insert-molecules -f newall.gro -ci $itp/tip3p.gro -nmol 90000 -o $a/mof_sol.gro -scale 0.45
!gmx insert-molecules -f $a/mof_sol.gro -ci $itp/K.gro -nmol 81 -o $a/mof_sol_K.gro  -scale 0.4
!gmx insert-molecules -f $a/mof_sol_K.gro -ci $itp/HCO3.gro -nmol 81 -o $a/mof_elec.gro  -scale 0.4
!gmx insert-molecules -f $a/mof_elec.gro -ci $itp/CO2.gro -nmol 54 -o $a/em.gro  -scale 0.4

viewgro(a+'/em' )

            :-) GROMACS - gmx insert-molecules, 2024.2-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2024.2/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2024.2
Working dir:  /Users/chenxili/GitHub/MOF_build_Demo/mof_build_demo
Command line:
  gmx insert-molecules -f newall.gro -ci MDdemo/MILitp/tip3p.gro -nmol 900 -o MDdemo/mdfiles/mof_sol.gro -scale 0.45

Reading solute configuration
Initialising inter-atomic distances...

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx insert-molecules uses the Van der Waals radii
from the source below. This

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:

itp = '../itp'
gmx_mpi insert-molecules -f newall.gro -ci ($itp)/tip3p.gro -nmol 90000 -o mof_sol.gro -scale 0.35
gmx_mpi insert-molecules -f mof_sol.gro -ci ($itp)/K.gro -nmol 81 -o mof_sol_K.gro  -scale 0.4
gmx_mpi insert-molecules -f mof_sol_K.gro -ci ($itp)/HCO3.gro -nmol 81 -o mof_elec.gro  -scale 0.4
gmx_mpi insert-molecules -f mof_elec.gro -ci($itp)/CO2.gro -nmol 54 -o em.gro  -scale 0.4

itp = '../itp'                                          
gmx_mpi insert-molecules -f newall.gro -ci    $itp/tip3p.gro -nmol 90000 -o mof_sol.gro -scale 0.35
gmx_mpi insert-molecules -f mof_sol.gro -ci   $itp/K.gro -nmol 81 -o mof_sol_K.gro  -scale 0.4
gmx_mpi insert-molecules -f mof_sol_K.gro -ci $itp/HCO3.gro -nmol 81 -o mof_elec.gro  -scale 0.4
gmx_mpi insert-molecules -f mof_elec.gro -ci. $itp/CO2.gro -nmol 54 -o em.gro  -scale 0.4

In [11]:
"""step3 generate topology file for MD simulation"""

import MDprepare.itp_process as ip
import os,re

middlelines, sectorname = ip.parsetop("MDdemo/MILitp/template.top")

res_num = (
    "AL6            396\nMOH            396\nCUT            292\n"
    + "TCP            125\nSOL            90000\nK              81\n"
    + "BCN            81\nCO2            54\n"
)


def top_gen(itp_file,middlelines,outtopname,input_res_num):
    with open(itp_file, "r") as f:
        lines = f.readlines()

    keyword1 = "atomtype"
    keyword2 = "moleculetype"
    for eachline in lines:  # search for keywords and get linenumber
        if re.search(keyword1, eachline):
            start = lines.index(eachline) + 2
        elif re.search(keyword2, eachline):
            end = lines.index(eachline) - 1

    target_lines = [line for line in lines[start:end] if line.strip()]

    newstart = end + 1

    #with open("MDdemo/itp/linker.itp", "w") as fp:
     #   fp.writelines(lines[newstart:])

    target_lines.append("\n")
    sec1 = target_lines
    sec2 = [
        #'#include "../itp/linker.itp"'
        #+ "\n"
        #+ '#include "../itp/tip3p.itp"'
        #+ "\n"
         '#include "../itp/CO2.itp"'
        + "\n"
        + "\n"
    ]
    sec3 = ["MOF_MIL" + "\n" + "\n"]
    sec4 = [
        "\n"+input_res_num
        + "\n"
        + "\n"
    ]
    newtop = (
        middlelines[0]
        + middlelines[1]
        + sec1
        + middlelines[2]
        + sec2
        + middlelines[3]
        + sec3
        + middlelines[4]
        + sec4
    )

    newpath = os.path.abspath ( '')+'/MDdemo/mdfiles/'   # input file
    os.makedirs(newpath,exist_ok=True)

    with open(newpath+outtopname, "w") as f:
        f.writelines(newtop)

top_gen("MDdemo/MILitp/newTCPP_lacvpss.itp", middlelines, "em.top", res_num)

In [4]:
!ls


MDdemo	   Residues  cut.xyz	       linker.xyz  testMIL54tetra.ipynb
MDprepare  all.gro   demo_UIO66.ipynb  newall.gro
MOF_build  all.xyz   itps	       node.xyz
